In [1]:
import torch 
import torch.nn as nn

# 二维互运算操作
def corr2d(X, K):
    H, W = X.shape  # 数组维度
    h, w = K.shape
    Y = torch.zeros(H-h+1, W-w+1)  # 计算结果数组维度
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i+h, j: j+w] * K).sum()
    return Y
    

In [5]:
# 查看下效果
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[2, 3]])
Y = corr2d(X, K)
print(Y)

tensor([[ 3.,  8.],
        [18., 23.],
        [33., 38.]])


In [3]:
# 构建二维卷积层:该层参数包括卷积核和标量偏置
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [21]:
# 构造一张6X8的图像，中间4列为黑（0），其余为白（1），希望检测到颜色边缘。
# 标签是一个6X7的二维数组，第2列是1（从1到0的边缘），第6列是-1（从0到1的边缘）
X = torch.ones(6, 8)
Y = torch.zeros(6, 7)
X[:, 2: 6] = 0
Y[:, 1] = 1
Y[:, 5] = -1
print(X)
print(Y)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [25]:
# 对以上自行构造的图像，希望用1X2的卷积层，学习检测边缘
conv2d = Conv2D(kernel_size=(1, 2))
step = 30
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()  # 损失值计算
    l.backward()
    # 优化器优化（梯度下降，此处无直接使用某个优化器）
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    
    # 梯度清零
    conv2d.weight.grad.zero_()
    conv2d.bias.grad.zero_()
    if (i + 1) % 5 == 0:  # 每5次迭代输出损失值观察
        print('Step %d, loss %.3f' % (i + 1, l.item()))
print(conv2d.weight.data)
print(conv2d.bias.data)

Step 5, loss 5.529
Step 10, loss 1.453
Step 15, loss 0.395
Step 20, loss 0.109
Step 25, loss 0.030
Step 30, loss 0.008
tensor([[ 0.9763, -0.9771]])
tensor([0.0005])


In [36]:
# pytorch简洁实现卷积层
# 主要用到nn模块的Conv2d类
X = torch.rand(4, 2, 3, 5)  # 批量大小4 通道数2 
conv2d = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=(3, 5), stride=1, padding=(1, 2))
Y = conv2d(X)
# 从Y输出结果看，由于out_channals=3致使通道数发生变化
# 由于卷积核大小、步幅和填充参数设置合理，输出与输入形状一致
print('Y.shape: ', Y.shape)  
print('weight.shape: ', conv2d.weight.shape)
print('bias.shape: ', conv2d.bias.shape)

Y.shape:  torch.Size([4, 3, 3, 5])
weight.shape:  torch.Size([3, 2, 3, 5])
bias.shape:  torch.Size([3])


In [38]:
# pytorch简洁实现池化层
# 主要用到nn模块的MaxPool2d最大池化层类
X = torch.arange(32, dtype=torch.float32).view(1, 2, 4, 4)  # 批量大小1 通道2  高宽4,4的图像数据
pool2d = nn.MaxPool2d(kernel_size=3, padding=1, stride=(2,1))
Y = pool2d(X)
print(X)
print(Y)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]]]])
tensor([[[[ 5.,  6.,  7.,  7.],
          [13., 14., 15., 15.]],

         [[21., 22., 23., 23.],
          [29., 30., 31., 31.]]]])
